Import the hoomd and other python packages.

In [1]:
import hoomd
import hoomd.md
import numpy as np
from matplotlib import pyplot 
import ase # Atomic simulation environment
import ase.io
from ase.visualize import view

In [2]:
hoomd.context.initialize('--mode=cpu');

HOOMD-blue v2.8.0-2-gff981a78f DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 
Compiled: 15/11/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


#### Define the simulation system, and multiply it 7 times - create 686 particles in a box, on a perfect lattice:

In [3]:
initial_cell = hoomd.lattice.unitcell(N = 2, # two atoms in the initial cell
                            a1 = [5,0,0],
                            a2 = [0,5,0],
                            a3 = [0,0,5], # these define a cubic box of length 3.0
                            position = [[0.0,0.0,0.0], [2.5, 2.5, 2.5]], # set positions for the two particles (they should not be placed on top of each other)
                            type_name = ['G1', 'G1'], # one of the atoms will be called type 'A', the other type 'B'
                            mass = [1.0, 1.0], # 
                            );
system=hoomd.init.create_lattice(initial_cell,n=7) 

notice(2): Group "all" created containing 686 particles


In [4]:
#Extract the HOOMD configuration in ASE format - I prefer this for visualisation 
def save_config(hoomd_system,atom_types=["C","F"]):
    
    lattice=np.array([hoomd_system.box.get_lattice_vector(i=i) for i in range(3)])
    
    pos=[system.particles[i].position for i in range(hoomd_system.particles.types.pdata.getN())]
    types=[atom_types[system.particles[i].type] for i in range(hoomd_system.particles.types.pdata.getN())]
    ase_atoms = ase.Atoms(pbc=[(True,True,True)],cell=lattice,positions=pos, symbols=types)
    
    ase_atoms.wrap()
    return ase_atoms

#### Initialise and set the Lennard-Jones interaction parameters, $\sigma$ and $\epsilon$, for all combinations of pair types. 

In [5]:
nl = hoomd.md.nlist.cell()
lj = hoomd.md.pair.lj(r_cut=2.5, nlist=nl) # define potential
lj.pair_coeff.set('G1', 'G1', epsilon=1.0, sigma=1.0)
lj.pair_coeff.set('G2', 'G2', epsilon=1.0, sigma=1.5)
lj.pair_coeff.set('G3', 'G3', epsilon=1.0, sigma=2.0)
# mixed type sigmas are set as the average:
lj.pair_coeff.set('G1', 'G2', epsilon=1.0, sigma=1.25)
lj.pair_coeff.set('G2', 'G3', epsilon=1.0, sigma=1.75)
lj.pair_coeff.set('G1', 'G3', epsilon=1.0, sigma=1.5)

###  Set up the MD simulation and the ensemble

Set of the MD, timestep and integrator:

In [7]:
all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0005);

We will save these properties during the simulations in different logfiles:

In [8]:
quantities=['volume','lx','ly','lz','potential_energy','kinetic_energy','temperature','pressure']

In [9]:
hoomd.analyze.log(filename="output_equilibration.log",
                  quantities=quantities,
                  period=100,
                  overwrite=True) # keep appending the existing file

Let's do a short run on the ($N$,$V$,$T$) ensemble: (T=0.8 in LJ units) 

In [10]:
nvt = hoomd.md.integrate.langevin(group=all, kT=0.8, seed=10)
hoomd.run(2000)

notice(2): integrate.langevin/bd is using specified gamma values
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 686
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 2000 / 2000 | TPS 3969.3 | ETA 00:00:00
Average TPS: 3960.25
---------
-- Neighborlist stats:
0 normal updates / 20 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 2 / n_neigh_avg: 0.0393586
shortest rebuild period: 100
-- Cell list stats:
Dimension: 12, 12, 12
n_min    : 0 / n_max: 2 / n_avg: 0.396991
** run complete **


In [11]:
snapshot = system.take_snapshot()
print("Current number of particles:",snapshot.particles.N)
n_part=snapshot.particles.N
print("Current types of particles:",snapshot.particles.typeid)

Current number of particles: 686
Current types of particles: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [12]:
# Add one partticle to the system (by default it will be type 0)
snapshot.particles.resize(n_part+1)

In [13]:
print("Default coordinates of new particle",snapshot.particles.position[n_part]) # n_part-th is the new particle (numbering starts at 0)
snapshot.particles.position[n_part] = [1.5,2.5,3.5] # set the positions of the new particle
print("Coordinets of the new particle",snapshot.particles.position[n_part])

Default coordinates of new particle [0. 0. 0.]
Coordinets of the new particle [1.5 2.5 3.5]


In [14]:
print(snapshot.particles.typeid[n_part])
# Redefine what the particle type is for the one we created: 
snapshot.particles.typeid[n_part]=0
# The above can be done for any other particle, we could just redifine their type? E.g:

#Time for a rrandom particle to move up to the next level of their livecycle:
random_part=np.random.randint(1,n_part+1)
snapshot.particles.typeid[random_part]=1

# Check:
print(snapshot.particles.typeid)

0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 

In [15]:
# Restore snapshot so we can continue simulationg the new system with the added particle:
system.restore_snapshot(snapshot)

In [16]:
# This might fail if there's an overlap of particles...etc. So the insertion of the new one has to be done carefully.
hoomd.run(100)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 687
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:18 | Step 2100 / 2100 | TPS 8557.98 | ETA 00:00:00
Average TPS: 7967.49
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 2 / n_neigh_avg: 0.0640466
shortest rebuild period: 100
-- Cell list stats:
Dimension: 12, 12, 12
n_min    : 0 / n_max: 2 / n_avg: 0.397569
** run complete **
